In [4]:
import pandas as pd
import numpy as np
solar_data = pd.read_csv('data/solarpower.csv')
solar_data.head()

,date,cum_power
0,2017-01-01,20111.0
1,2017-01-02,20112.0
2,2017-01-03,20115.0
3,2017-01-04,20117.0
4,2017-01-05,20119.0


In [5]:
# Replace -1 with NaN
solar_data['cum_power'] = solar_data['cum_power'].replace(-1, np.nan)

# Fill NaN values with the average of the previous and next values
solar_data['cum_power'] = solar_data['cum_power'].interpolate()

# Calculate the mean of the cum_power column
mean_cum_power = solar_data['cum_power'].mean()

# Print the mean rounded to the nearest integer
print(round(mean_cum_power))

23848


In [7]:
# Create the day_power column (daily power generation)

# Convert 'date' to datetime
solar_data['date'] = pd.to_datetime(solar_data['date'])

# Sort by date
solar_data = solar_data.sort_values(by='date')

# Create the day_power column (daily power generation)
solar_data['day_power'] = solar_data['cum_power'].diff()

# Fill any remaining NaN values in day_power with the mean of day_power
solar_data['day_power'] = solar_data['day_power'].fillna(solar_data['day_power'].mean())

# Extract year-month period from the date
solar_data['period'] = pd.to_datetime(solar_data['date']).dt.to_period('M')

# Group by period and calculate the mean of day_power
monthly_avg_day_power = solar_data.groupby('period')['day_power'].mean()

# Get the average daily power generation for March 2017 and round to the nearest integer
march_2017_avg_day_power = round(monthly_avg_day_power['2017-03'])

print(march_2017_avg_day_power)

11


In [8]:
import plotly.express as px

# Group by period and calculate the mean of day_power
monthly_avg_day_power = solar_data.groupby('period')['day_power'].mean().reset_index()

# Convert period to string for plotting
monthly_avg_day_power['period'] = monthly_avg_day_power['period'].astype(str)

# Plot the time series using plotly
fig = px.line(monthly_avg_day_power, x='period', y='day_power', title='Average Daily Solar Power Generation per Month-Year',
              labels={'period': 'Month-Year', 'day_power': 'Average Daily Power Generation (kWh)'})

fig.show()